In [ ]:
import pandas as pd
#from IPython import get_ipython
#get_ipython().run_line_magic('matplotlib', 'widget ')
import matplotlib.pyplot as plt

In [11]:
df = pd.read_csv('data/40014.csv')

In [12]:
# Filter the email messages
print(df.shape)
# Remove rows with more than 80% of missing values
df = df.dropna(thresh=0.2 * len(df.columns), axis=0)
# Remove columns with more than 90% of missing values
df = df.dropna(thresh=0.1 * len(df), axis=1)
print(df.shape)

(1014838, 48)
(926782, 48)


In [22]:
# Remove duplicates
print(df.shape)
df = df.drop_duplicates(subset=['timestamp'])
print(df.shape)

(926782, 48)
(906562, 48)


In [29]:
# get timestamp and battery voltage
battery_voltage = df['nice_battery_mv'].values
UTC_time = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S', utc=True)
UNIX_time = UTC_time.astype('int64') // 10**9

In [ ]:
# eclipse file 2
eclipse_file = '../data/eclipse/eclipses.txt'

df = pd.read_csv(eclipse_file, sep=",", skiprows=1, header=None, names=["Start Time (UTC)", "Stop Time (UTC)"])

# Convert the columns to datetime
df["Start Time (UTC)"] = pd.to_datetime(df["Start Time (UTC)"].str.strip())
df["Stop Time (UTC)"] = pd.to_datetime(df["Stop Time (UTC)"].str.strip())

timestamps_start_event = ((df['Start Time (UTC)'].dt.tz_localize(None) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s'))
timestamps_end_event = ((df['Stop Time (UTC)'].dt.tz_localize(None) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s'))

time_s_start_event = np.zeros(int(len(timestamps_start_event) * 2) - 1)
time_s_end_event = np.zeros(int(len(timestamps_end_event) * 2) - 1)
eclipse_event_type = np.zeros(int(len(timestamps_end_event) * 2) - 1)
for i in range(0, len(timestamps_start_event), 1):
    time_s_start_event[i*2] = timestamps_start_event[i]
    time_s_end_event[i*2] = timestamps_end_event[i]
    eclipse_event_type[i*2] = 1

    if i + 1 != len(timestamps_start_event):
        time_s_start_event[i*2+1] = timestamps_end_event[i]
        time_s_end_event[i*2+1] = timestamps_start_event[i + 1]
        eclipse_event_type[i*2+1] = 0

In [ ]:
# Make the plot with eclipse
eclipse_points_x = np.zeros(len(time_s_start_event) * 2)
eclipse_points_y = np.zeros(len(time_s_end_event) * 2)
for i in range(len(time_s_start_event)):
    if eclipse_event_type[i] == 0:
        eclipse_points_y[2 * i] = 0
        eclipse_points_y[2 * i + 1] = 0
    elif eclipse_event_type[i] == 1:
        eclipse_points_y[2 * i] = 1
        eclipse_points_y[2 * i + 1] = 1
    
    eclipse_points_x[2 * i] = time_s_start_event[i]
    eclipse_points_x[2 * i + 1] = time_s_end_event[i]


datetime_objects = pd.to_datetime(eclipse_points_x, unit='s')
df = pd.DataFrame({'datetime': datetime_objects})
df['formatted_datetime'] = df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')


# Plot of each variable in time
for i, each in enumerate(names):
    fig, ax1 = plt.subplots(figsize=(15, 5))

    ax1.scatter(time_UTC, series[:, i], color='b', label=each, s=0.5)
    ax1.set_xlabel('Time UTC')
    ax1.set_ylabel(each, color='b')
    ax1.tick_params('y', colors='b')
    ax1.grid(True)

    ax2 = ax1.twinx()
    ax2.plot(datetime_objects, eclipse_points_y, label='Eclipse Points', color='r')
    ax2.set_ylabel('Eclipse Points', color='r')
    ax2.tick_params('y', colors='r')

    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

    plt.title(each)
    plt.show()


In [ ]:
# phase-plot

time_UNIX = np.array([dt.timestamp() for dt in time_UTC])

data_period = []
time_period = []
actual_time = []
period_l = []
period_count = []

for i in range(0, len(time_s_start_event)-1, 2):

    points_data = series[(time_UNIX > time_s_start_event[i]) & (time_UNIX < time_s_end_event[i+1]), :]
    points_time = time_UNIX[(time_UNIX > time_s_start_event[i]) & (time_UNIX < time_s_end_event[i+1])]
    period_length = time_s_end_event[i+1] - time_s_start_event[i]

    for j in range(len(points_data)):
        current_time = points_time[j] - time_s_start_event[i]

        time_period.append(current_time/period_length)
        # data_period.append(points_data[j])

        data_period.append(points_data[j])
        actual_time.append(points_time[j])
        period_l.append(period_length)
        period_count.append(i)

In [ ]:
# Convert 1 Jan 2020 to UNIX time
start_time = pd.Timestamp("2020-01-01").timestamp()
end_time = pd.Timestamp("2021-01-01").timestamp()   

start_time2 = start_time - 86400 # day
end_time2 = end_time + 86400

# Filter
ndata_period = np.array(data_period)
ntime_period = np.array(time_period)
nactual_time = np.array(actual_time)
nperiod_l = np.array(period_l)
nperiod_count = np.array(period_count)
neclipse_points_x = np.array(eclipse_points_x)
neclipse_points_y = np.array(eclipse_points_y)

ndata_period = ndata_period[(nactual_time > start_time) & (nactual_time < end_time)]
ntime_period = ntime_period[(nactual_time > start_time) & (nactual_time < end_time)]
nperiod_l = nperiod_l[(nactual_time > start_time) & (nactual_time < end_time)]
nperiod_count = nperiod_count[(nactual_time > start_time) & (nactual_time < end_time)]
nactual_time = nactual_time[(nactual_time > start_time) & (nactual_time < end_time)]
neclipse_points_y = neclipse_points_y[(neclipse_points_x > start_time2) & (neclipse_points_x < end_time2)]
neclipse_points_x = neclipse_points_x[(neclipse_points_x > start_time2) & (neclipse_points_x < end_time2)]

In [ ]:
import scipy.io
# save the data
scipy.io.savemat('data_phase.mat', {
    'data': ndata_period,
    'time': ntime_period,
    'actual_time': nactual_time,
    'period_length': nperiod_l,
    'period_count': nperiod_count,
    'eclipse_points_x': neclipse_points_x,
    'eclipse_points_y': neclipse_points_y
})